In [ ]:
from typing import List, Dict, Any

from pathlib import Path
import os, sys
import time
import datetime as dt

import pandas as pd
import ray
from ray import train, tune

from recommenders.evaluation.python_evaluation import metrics as metrics_dict

from recsys24_daos.model_selection import cvtt_open
from recsys24_daos.datasets import to_microsoft, filter_window_size
from recsys24_daos.models import LightGCNCustom
import recsys24_daos.utils.notebooks as nbu

import recommenders
if recommenders.__version__ == '1.2.0':
    print("Ignoring warnings")
    import warnings
    warnings.simplefilter(action='ignore', category=FutureWarning)

import paths

nbu.print_versions('ray', 'tensorflow')

In [ ]:
# Others config
SEED: int = 57
RAY_RESULTS_PATH: Path = '~/ray_results3.11'

# Dataset splits config
ORG_NAME = 'Decentraland'
SPLITS_FREQ = 'W-THU'  # Split weekly
LAST_SPLITS = 10  # Use just last 10 splits
SPLITS_NORMALIZE = True

# Training config
SMALL_EXPERIMENT: bool = os.uname().nodename != 'lamarck'
MAX_EPOCHS: int = 200
EPOCHS_PER_ITER: int = 5
SAMPLES_PER_SPLIT: int = 100
OPTIM_METRIC: str = 'map@10'

# Search space config
MAX_EMBEDDING_DIM = 1024
MAX_BATCH_SIZE = 10 # 2**10
MIN_LR = 1e-4
WINDOW_SIZES = ['7d', '14d', '21d', '30d', '60d', '90d', '10Y']
GPUS = 16

# Eval config
K_RECOMMENDATIONS: List[int] = [1,3,5,10,15,100]
METRICS: List[str] = ["recall", "ndcg", "precision", "map"]

In [ ]:
RAY_RESULTS_PATH = Path(RAY_RESULTS_PATH).expanduser()

## Obtain dataset

In [ ]:
!pwd

In [ ]:
dfp = paths.load_proposals(ORG_NAME)
dfv = paths.load_votes(ORG_NAME)

print(dfp.info())
print(dfv.info())

## Split data

In [ ]:
df = to_microsoft(dfv)
df

In [ ]:
folds = list(cvtt_open(df, SPLITS_FREQ, dfp, remove_not_in_train_col='userID'))
print(len(folds), "folds")
folds = folds[-LAST_SPLITS:]
for i, (dftrain, dftest, t, open_proposals) in enumerate(folds):
    min_train = dftrain['timestamp'].min().date()
    max_train = dftrain['timestamp'].max().date()
    min_test  = dftest['timestamp'].min().date()
    max_test  = dftest['timestamp'].max().date()

    train_users = len(set(dftrain['userID']))
    test_users = len(set(dftest['userID']))
    
    print(f"Split {i}, train from: {min_train} to {max_train}, test from: {min_test} to {max_test}")
    print(f"  t: {t}")
    print(f"  open proposals: {len(open_proposals)}")
    print(f"  len(train): {len(dftrain)}, len(test): {len(dftest)}")
    print(f"  users(train): {train_users}, users(test): {test_users}")

    print()
    dftest['prediction'] = 1
    for m in METRICS:
        f = metrics_dict[f'{m}_at_k']
        print(f"  highest possible {m}@{K_RECOMMENDATIONS[0]}:\t{f(dftest, dftest, k=K_RECOMMENDATIONS[0], relevancy_method='top_k'):.4f}")

    print("-"*30)

## Testing model

In [ ]:
from recommenders.models.deeprec.DataModel.ImplicitCF import ImplicitCF
from recommenders.models.deeprec.deeprec_utils import prepare_hparams

In [ ]:
hparams = prepare_hparams(
    model_type='lightgcn',
    n_layers=3,
    batch_size=512,
    embed_size=64,
    epochs=2,
    learning_rate=0.001,
    decay=0.001,
    metrics=["recall", "ndcg", "precision", "map"],
    eval_epoch=2,
    top_k=K_RECOMMENDATIONS[0],
    save_model=False,
    MODEL_DIR='./data/model/lightgcn/',
)
dataloader = ImplicitCF(train=folds[-1][0], test=folds[-1][1], seed=SEED)
print("items:", dataloader.n_items, "user:", dataloader.n_users)
model = LightGCNCustom(data=dataloader, hparams=hparams)

In [ ]:
model.fit()
model.run_eval()

In [ ]:
model.recommend_k_items(
    dataloader.test, 
    top_k=3, 
    use_id=True, 
    remove_seen=True, 
    recommend_from=folds[-1][3]
)

## Defining trainable

In [ ]:
from recommenders.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k, r_precision_at_k

In [ ]:
class TrainLightGCN(tune.Trainable):
    def setup(
        self,
        config: Dict[str, Any],
        data,
    ):
        self.config = config

        self.hparams = prepare_hparams(
            model_type='lightgcn',
            n_layers=config['conv_layers'],
            batch_size=2**config['batch_size'],
            embed_size=config['embedding_dim'],
            epochs=EPOCHS_PER_ITER,
            learning_rate=config['learning_rate'],
            decay=config['l2'],
            metrics=METRICS,
            eval_epoch=-1,
            top_k=K_RECOMMENDATIONS[0],
            save_model=False,
            MODEL_DIR='./data/model/lightgcn/',
        )

        train, test, self.t, self.open_proposals = data
        train_filtered = filter_window_size(train, self.t, config['window_size'])
        self.dataloader = ImplicitCF(train=train_filtered, test=test, seed=SEED)
        self.model = LightGCNCustom(self.hparams, self.dataloader, seed=SEED)
        self.total_train = 0
        self.total_eval = 0

    @property
    def iteration(self):
        return self.model.epochs_done

    @property
    def training_iteration(self):
        return self.model.epochs_done

    def step(self):
        """
        As a rule of thumb, the execution time of step should be large enough to avoid overheads 
        (i.e. more than a few seconds), but short enough to report progress periodically 
        (i.e. at most a few minutes).
        """
        assert EPOCHS_PER_ITER > 0

        train_start = time.time()
        for _ in range(EPOCHS_PER_ITER):
            ret = self.model.fit_epoch()
        eval_start = train_end = time.time()

        eval_dict = {'model_'+k:v for k,v in zip(self.model.metrics, self.model.run_eval())}
        for k in K_RECOMMENDATIONS:
            recs = self.model.recommend_k_items(
                self.dataloader.test, 
                top_k=k,
                use_id=True, 
                remove_seen=True, 
                recommend_from=self.open_proposals,
            )
            
            eval_dict[f'precision@{k}'] = precision_at_k(self.dataloader.test, recs, k=k)
            eval_dict[f'ndcg@{k}'] = ndcg_at_k(self.dataloader.test, recs, k=k)
            eval_dict[f'recall@{k}'] = recall_at_k(self.dataloader.test, recs, k=k)
            eval_dict[f'map@{k}'] = map_at_k(self.dataloader.test, recs, k=k)
            eval_dict[f'rprecision@{k}'] = r_precision_at_k(self.dataloader.test, recs, k=k)

        eval_end = time.time()

        self.total_train += train_end - train_start
        self.total_eval += eval_end - eval_start
        
        return {
            'iteration': self.iteration,
            'loss': ret[0],
            'mf_loss': ret[1],
            'emb_loss': ret[2],
            **eval_dict,
            'time_train': train_end-train_start,
            'time_test': eval_end-eval_start,
            'time_total_train': self.total_train,
            'time_total_test': self.total_eval,
        }

    def save_checkpoint(self, checkpoint_dir):
        checkpoint_path = os.path.join(checkpoint_dir, "model")
        self.model.saver.save(
            sess=self.model.sess,
            save_path=checkpoint_path,
        )
        return checkpoint_dir

    def load_checkpoint(self, checkpoint_path):
        self.model.load(checkpoint_path)

## Big experiment

In [ ]:
RAY_RESULTS_PATH

In [ ]:
print(os.uname().nodename)

### SET TRAINING RESOURCES
if os.uname().nodename == 'lamarck':
    # assert torch.cuda.is_available()

    NUM_SAMPLES = SAMPLES_PER_SPLIT
    # Every run takes approx half a gig of vram (no optimizations)
    # The RTX 4090 has 24GB so we can run the model about 48 times
    resources_per_trial={
        'cpu': 1,
        'gpu': 1 / GPUS,
    }
else:
    NUM_SAMPLES = 1
    resources_per_trial={
        'cpu': 1,
        # It takes about 1.5 GiB with full training data, but I put a bit more because
        # this notebook also takes a bit of memory
        'memory': 2e9,
    }
print(resources_per_trial)

In [ ]:
from ray.tune.search.hyperopt import HyperOptSearch

In [ ]:
def getTunerOnFold(f, points_to_evaluate = None):    
    name = paths.lightgcn_ray_tune_fname(ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE, OPTIM_METRIC, fold=f)
    experiments = list(RAY_RESULTS_PATH.glob(f'{name}_*'))
    last_experiment = max(experiments, key=lambda x: x.stat().st_ctime) if experiments else None

    dftrain,dftest,t,open_proposals = folds[f]
    param_space = dict(
        fold=f,
        batch_size=tune.randint(6, MAX_BATCH_SIZE), # 64 - 1024
        embedding_dim=tune.lograndint(1, MAX_EMBEDDING_DIM, base=2),
        conv_layers=tune.randint(1,4),
        learning_rate=tune.qloguniform(MIN_LR, 1, 1e-4),
        l2=tune.loguniform(1e-7, 1e-2, 1e-7),
        window_size=tune.choice(WINDOW_SIZES),
    )
    
    ### RESTORE EXPERIMENT OR CREATE A NEW ONE
    if last_experiment and tune.Tuner.can_restore(last_experiment):
        print(f"Restoring last experiment: {last_experiment}")
        tuner = tune.Tuner.restore(
            str(last_experiment),
            trainable=tune.with_resources(
                # tune.with_parameters(TrainLightGCN,  train=dftrain, test=dftest, open_proposals=open_proposals),
                tune.with_parameters(TrainLightGCN, data=folds[f]),
                resources_per_trial,
            ),
            restart_errored=True,
            param_space=param_space,
        )
    else:
        print(f"No experiment found for fold {f}, creating new tuner with {NUM_SAMPLES} samples")
        search_alg = None
        search_alg = HyperOptSearch(
            # points_to_evaluate=[{
            #     'batch_size': 8, # 2**8 = 256
            #     'learning_rate': 10e-2,
            #     'l2': 10e-6,
            #     'embedding_dim': 100,
            #     'conv_layers': 3,
            # }],
            points_to_evaluate = points_to_evaluate,
            random_state_seed=SEED,
        )
        # search_alg = tune.search.Repeater(search_alg, N_SPLITS-SKIP_SPLIT)
        
        tuner = tune.Tuner(
            tune.with_resources(
                # tune.with_parameters(TrainLightGCN,  train=dftrain, test=dftest, open_proposals=open_proposals),
                tune.with_parameters(TrainLightGCN, data=folds[f]),
                resources_per_trial,
            ),
            run_config=train.RunConfig(
                stop={'training_iteration': MAX_EPOCHS/EPOCHS_PER_ITER, 'time_total_train': 300},
                name=name + f'_{dt.datetime.now().isoformat()}',
                storage_path=RAY_RESULTS_PATH,
                # failure_config=train.FailureConfig(fail_fast='raise'),
                failure_config=train.FailureConfig(max_failures=3),
            ),
            param_space=param_space,
            tune_config=tune.TuneConfig(
                search_alg=search_alg,
                num_samples=NUM_SAMPLES,
                metric=OPTIM_METRIC,
                mode='max',
            )
        )

    return tuner

In [ ]:
import logging

def findConfig(rg):
    for r in rg:
        if r.config:
            if all((r.config[k] == v for k, v in last_best_result.config.items() if k != 'fold')):
                return r

    return None

tuners = []
results = []
last_best_result = None
for f in range(LAST_SPLITS):
    best_prev_config = None
    if last_best_result is not None:
        best_prev_config = last_best_result.config.copy()
        best_prev_config['fold'] += 1
        best_prev_config = [best_prev_config]
    
    t = getTunerOnFold(f, best_prev_config)
    tuners.append(t)

    rg = t.fit()
    assert rg.num_errors == 0, f"There are {rg.num_errors} errors"
    assert rg.num_terminated >= NUM_SAMPLES, f'Some samples are not terminated ({rg.num_terminated} != {NUM_SAMPLES})'
    results.append(rg)

    # Assert that the prev config has been tried
    if last_best_result is not None:
        # if not any( 
        #     all((r.config[k] == v for k, v in last_best_result.config.items() if k != 'fold'))
        #     for r in rg if r.config
        # ):
        if not findConfig(rg):
            print("Best config:", last_best_result.config)
            assert False, f"The best config from previous fold has not been tested in fold {f}"    
        else:
            logging.info(f'Fold {f}. Best prev result was {last_best_result.path} and config has been found {findConfig(rg).path}')
    
    last_best_result = rg.get_best_result()

    print(f"Finished training for fold {f}")